In [11]:
import chit

%load_ext autoreload
%autoreload 2

chat = chit.Chat.clone("chitimps.json")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
chat.commit("^N/$jupyter")

Textarea(value='', layout=Layout(height='200px', width='100%'), placeholder='Type your message here...')

In [6]:
chat.commit("^$N/code")

In [13]:
chat.checkout(0)
chat.branch("globals", checkout=True)

In [15]:
chat.commit(
"""
I have written a Python package, `chit` which has some of its settings as global constants:

VERBOSE = True

FORCE = False

AUTOSAVE = True

EDITOR = "code"

If I import this package as import chit and write `chit.FORCE = True`, this does not however seem to affect chit.FORCE in the module itself. For example that module contains a function:

def cconfirm(prompt: str) -> bool:
    print(FORCE)
    if not FORCE:
        response = input(f"{prompt} (y/n) ")
        return response.lower() == "y"
    return True

Where I added the print(FORCE) as a debugging statement. When I run this function, it always prints False, even if I have set chit.FORCE = True. Why is this?
"""
)

chat.commit()

This behavior occurs because Python modules maintain their own namespace, and when you import variables from a module, you're actually getting a reference to those variables. However, when you modify the variable through the module reference (chit.FORCE), you're only modifying your local reference to that variable, not the original variable in the module's namespace.

To make this work as you expect, you have a few options:

1. Use a configuration class or object:
```python
# In your module (chit.py)
class Config:
    VERBOSE = True
    FORCE = False
    AUTOSAVE = True
    EDITOR = "code"

config = Config()

def cconfirm(prompt: str) -> bool:
    print(config.FORCE)
    if not config.FORCE:
        response = input(f"{prompt} (y/n) ")
        return response.lower() == "y"
    return True
```

Then you can modify it like:
```python
import chit
chit.config.FORCE = True
```

2. Use a module-level dictionary:
```python
# In your module (chit.py)
settings = {
    'VERBOSE': True,
    'FOR

In [16]:
chat.commit(
"""
No, I think you're wrong -- setting chit.FORCE = True should affect the module itself (whereas if I imported `from chit import FORCE` and set `FORCE = True`, that would not affect the module).

I think the problem is that FORCE is not actually in `chit`, but in `chit/chit.py` and it's being imported in `__init__` as `from chit import *`.
"""
)
chat.commit()

Ah, you're absolutely right - I apologize for my confusion! Yes, if you use `import chit` and then `chit.FORCE = True`, it should indeed modify the module's variable.

And yes, your diagnosis is correct: the issue is with the way the constants are being imported. When you do `from chit import *` in `__init__.py`, you're creating new references to those variables in the `__init__.py` namespace, and those are what get exposed when someone imports the package.

To fix this, you could either:

1. Move the constants to `__init__.py` directly, or

2. In `__init__.py`, instead of using `from chit import *`, explicitly import and expose the constants:
```python
from .chit import FORCE, VERBOSE, AUTOSAVE, EDITOR
```

or

3. In `__init__.py`, import the module itself:
```python
from . import chit
FORCE = chit.FORCE
VERBOSE = chit.VERBOSE
AUTOSAVE = chit.AUTOSAVE
EDITOR = chit.EDITOR
```

Option 1 or 2 would probably be the cleanest solutions. The star import (`from chit import *`) is generally d

In [17]:
chat.commit("I don't think 2 or 3 would work either.")
chat.commit()

You're right, I apologize. Even with explicit imports (options 2 and 3), you would still be creating new references in `__init__.py` rather than exposing the original variables.

The solution would be to either:

1. Move the constants directly to `__init__.py`, or

2. Import and expose the submodule itself in `__init__.py`:
```python
from . import chit
```

Then users would need to use `chit.chit.FORCE`, which isn't ideal.

Moving the constants to `__init__.py` is probably the cleanest solution if you want users to be able to modify them through `chit.FORCE`.

In [19]:
chat.log()

[S_]0a2fae4b──[U_]0c718877──[A_]4b63df07──[U_]dc7df04f──[A_]f5eb2cb2──[U_]7fed9b3c──[A_]ef9cb79c── (master)
            │                                                       └─[U_]057e186c──[A_]aad7afbd──[U_]1557f58b──[A_]6b2686b6──[U_]c4820bf0──[A_]11e4c7a4──[U_]23b84bfb──[A_]ee07b62c──[U_]f4f5a248──[A_]e12a8f5f──[U_]add7cb48──[A_]ba0beafb──[U_]2cae65c6──[A_]de4fda97──[U_]3ebbc548──[A_]fd4cda1c──[U_]2eac5cf6──[A_]da112702──[U_]83d4bee9──[A_]bd4cec31── (vim)
            └─[U_]c19adb91──[A_]fabd542b──[U_]9ffebb3f──[A_]7e83d1f1──[U_]cbdbaaec──[A*]3fc062cc── (globals*)
